In [2]:
from typing import Iterable, List, Dict, Optional

class NaryTree:
    def __init__(self, nodes: Iterable[str], bf: int = 2):
        self._nodes: List[str] = list(nodes)
        self._bf: int = bf
        self._node_to_idx: Dict[str, int] = {node: e for e, node in enumerate(self._nodes)}
        self._locked_nodes: Dict[str, int] = {}
        self._descendents_user_id_mapping: Dict[str, Dict[int, int]] = {}
        
    def parent_of(self, idx: int) -> int:
        return (idx - 1) // self._bf
        
    def first_child_of(self, idx: int) -> int:
        return idx * self._bf + 1
        
    def _descendents_of(self, node_idx: int) -> Iterable[str]:
        first_child = self.first_child_of(node_idx)
        for idx in range(
            first_child,
            min(len(self._nodes), first_child+self._bf)
        ):
            yield self._nodes[idx]
            yield from self._descendents_of(idx)
    
    def descendents_of(self, node_id: str) -> Iterable[str]:
        node_idx = self._node_to_idx.get(node_id)
        if node_idx is None:
            return
        yield from self._descendents_of(node_idx)
            
    def ancestors_of(self, node_id: str) -> Optional[Iterable[str]]:
        node_idx = self._node_to_idx.get(node_id)
        if node_idx is None:
            return
        if node_idx <= 0:
            return
        idx = self.parent_of(node_idx)
        while idx >= 0:
            yield self._nodes[idx]
            idx = self.parent_of(idx)
            
    def _add_user_id_to_descendents_mapping(self, node_id: str, user_id: int):
        count_mapping = self._descendents_user_id_mapping.get(node_id)
        if count_mapping is None:
            self._descendents_user_id_mapping[node_id] = {user_id: 1}
        elif user_id in count_mapping:
            count_mapping[user_id] += 1
        else:
            count_mapping[user_id] = 1
            
    def _remove_user_id_from_descendents_mapping(self, node_id: str, user_id: int):
        count_mapping = self._descendents_user_id_mapping.get(node_id)
        if count_mapping is None or user_id not in count_mapping:
            return
        count_mapping[user_id] -= 1
        if count_mapping[user_id] <= 0:
            count_mapping.pop(user_id)
        if len(count_mapping) == 0:
            self._descendents_user_id_mapping.pop(node_id)
        
    def _lock(self, node_id: str, user_id: int):
        for ancestor in self.ancestors_of(node_id):
            self._add_user_id_to_descendents_mapping(ancestor, user_id)
        self._locked_nodes[node_id] = user_id
    
    def lock(self, node_id: str, user_id: int) -> bool:
        if node_id in self._locked_nodes:
            return False
            
        for ancestor in self.ancestors_of(node_id):
            if ancestor in self._locked_nodes:
                return False
                
        if node_id in self._descendents_user_id_mapping:
            return False
            
        self._lock(node_id, user_id)
        return True
        
    def _unlock(self, node_id: str, user_id: int):
        for ancestor in self.ancestors_of(user_id):
            self._remove_user_id_from_descendents_mapping(ancestor, user_id)
        self._locked_nodes.pop(node_id)
    
    def unlock(self, node_id: str, user_id: int) -> bool:
        uid = self._locked_nodes.get(node_id)
        if uid is None or uid != user_id:
            return False
        self._unlock(node_id, user_id)
        return True
        
    def upgrade(self, node_id: str, user_id: int) -> bool:
        if node_id in self._locked_nodes:
            return False
            
        desendent_user_ids_mapping = self._descendents_user_id_mapping.get(node_id)
        if (
            (desendent_user_ids_mapping is None)
            or (len(desendent_user_ids_mapping) != 1)
            or (user_id not in desendent_user_ids_mapping)
        ):
            return False
            
        for descendent in self.descendents_of(node_id):
            if descendent in self._locked_nodes:
                self._unlock(descendent, user_id)
            
        self._lock(node_id, user_id)
        return True

def solve(m, nodes, queries):
    tree = NaryTree(nodes, m)
    for q in queries:
        q = q.split()
        op_code, node_id, user_id = q[0], q[1], int(q[2])
        if op_code == "1":
            b = tree.lock(node_id, user_id)
        elif op_code == "2":
            b = tree.unlock(node_id, user_id)
        elif op_code == "3":
            b = tree.upgrade(node_id, user_id)
        yield "true" if b else "false"
    

In [3]:
from pathlib import Path
import numpy as np
from string import ascii_lowercase

In [4]:
ascii_lowercase_arr = np.array(list(ascii_lowercase))

In [5]:
def random_string(size: int) -> str:
    return "".join(
        ascii_lowercase_arr
        .take(np.random.random_integers(0, len(ascii_lowercase) - 1, size=size))
    )

In [6]:
def random_strings(n, size):
    seen_before = set()
    i = 0
    while i < n:
        s = random_string(size)
        if s in seen_before:
            continue
        seen_before.add(s)
        yield s
        i += 1

In [7]:
def mk_random_queries(nodes, num_queries):
    op_codes = np.random.randint(low=1, high=4, size=num_queries)
    random_nodes = np.random.choice(nodes, size=num_queries)
    user_ids = np.random.randint(low=1, high=1001, size=num_queries)
    return [
        f"{op_code} {random_node} {user_id}"
        for op_code, random_node, user_id in zip(op_codes, random_nodes, user_ids)
    ]

In [8]:
def make_question_input(m, nodes, queries):
    lines = []
    lines.append(f"{len(nodes)}")
    lines.append(str(m))
    lines.append(f"{len(queries)}")
    for node in nodes:
        lines.append(str(node))
    for q in queries:
        lines.append(str(q))
    return "\n".join(lines)

In [9]:
def save_question_expected_output(inputs_folder: Path, outputs_folder: Path, branching_factor, nodes, queries):
    inputs_folder.mkdir(parents=True, exist_ok=True)
    outputs_folder.mkdir(parents=True, exist_ok=True)
    with open(inputs_folder / "input.txt", "w+") as fp:
        fp.write(make_question_input(branching_factor, nodes, queries))
        fp.write("\n")
    with open(outputs_folder / "output.txt", "w+") as fp:
        fp.write("\n".join(solve(branching_factor, nodes, queries)))
        fp.write("\n")

In [11]:
from pathlib import Path

root = Path("./")
root.mkdir(exist_ok=True)

for sample_id in range(100):
    num_nodes = np.random.randint(5, 1001)
    branching_factor = np.random.randint(2, 11)
    nodes = list(random_strings(num_nodes, 5))
    num_queries = np.random.randint(5, 1001)
    queries = mk_random_queries(nodes, num_queries)
    inputs_folder = root / f"test_case_{sample_id}"
    outputs_folder = root / f"test_case_{sample_id}"
    save_question_expected_output(inputs_folder, outputs_folder, branching_factor, nodes, queries)
    print(f"Done with generating sample: {sample_id}")
    

/home/ujjawalsinha/miniconda3/envs/py376/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: This function is deprecated. Please call randint(0, 25 + 1) instead
  after removing the cwd from sys.path.


Done with generating sample: 0
Done with generating sample: 1
Done with generating sample: 2
Done with generating sample: 3
Done with generating sample: 4
Done with generating sample: 5
Done with generating sample: 6
Done with generating sample: 7
Done with generating sample: 8
Done with generating sample: 9
Done with generating sample: 10
Done with generating sample: 11
Done with generating sample: 12
Done with generating sample: 13
Done with generating sample: 14
Done with generating sample: 15
Done with generating sample: 16
Done with generating sample: 17
Done with generating sample: 18
Done with generating sample: 19
Done with generating sample: 20
Done with generating sample: 21
Done with generating sample: 22
Done with generating sample: 23
Done with generating sample: 24
Done with generating sample: 25
Done with generating sample: 26
Done with generating sample: 27
Done with generating sample: 28
Done with generating sample: 29
Done with generating sample: 30
Done with generati